In [1]:
%load_ext lab_black
%load_ext autoreload
%autoreload 2

In [28]:
from pathlib import Path
from puzzle import Puzzle, Octopus

In [29]:
puz = Puzzle("tests.txt")
puz

Puzzle(fname='tests.txt')

In [30]:
puz.part_1()

1656

In [27]:
tmp = puz.octopi[9]
tmp.neighbors

[Octopus(row=0, col=8, nrg_level=2),
 Octopus(row=1, col=8, nrg_level=1),
 Octopus(row=1, col=9, nrg_level=1)]

In [21]:
tmp.row, tmp.col

(0, 9)

In [24]:
tmp.get_neighbors(puz.octopi)

[Octopus(row=0, col=8, nrg_level=2),
 Octopus(row=1, col=8, nrg_level=1),
 Octopus(row=1, col=9, nrg_level=1)]

In [ ]:
def validate(my_string):
    brackets = ["()", "{}", "[]", "<>"]
    while any(pair in my_string for pair in brackets):
        for br in brackets:
            my_string = my_string.replace(br, "")
    incomplete = set(my_string) - set("({[<") == set()
    invalid = [my_string.find(rt_br) for rt_br in ")}]>"]
    invalid = [x for x in invalid if x != -1]
    if invalid:
        invalid = min(invalid)
    else:
        invalid = None
    return my_string, incomplete, my_string[invalid]


Navigation("{([(<{}[<>[]}>{[]{[(<()>")

In [ ]:
my_string = "<"
bool(set(my_string) & set("({[<"))  # == set()

In [ ]:
validate("[[<[([]))<([[{}[[()]]]")

In [ ]:
validate("[({(<(())[]>[[{[]{<()<>>")

In [ ]:
"asdhf".find()

In [ ]:
fname = "tests.txt"
raw = Path(fname).open().readlines()
grid = np.array([list(row.strip()) for row in raw]).astype(int)


low_pts = []

for rownum, row in enumerate(grid):
    for colnum, val in enumerate(row):
        pt = Point(rownum, colnum, grid)
        if pt.is_lowest():
            low_pts.append(pt)
pt

In [ ]:
basins = np.where(grid == 9, 0, 1)
basins

In [ ]:
from scipy.ndimage import measurements

lw, num = measurements.label(basins)
area = measurements.sum(basins, lw, index=np.arange(lw.max() + 1))
area

## Black format my final answer

In [32]:
from pathlib import Path
from dataclasses import dataclass, field
from statistics import median


T1_ANS = 1656
T2_ANS = 195


@dataclass
class Octopus:
    row: int
    col: int
    nrg_level: int
    has_fired: bool = field(default=False, repr=False)
    neighbors: list = field(default_factory=list, repr=False)
    total_flashes: int = 0

    def __post_init__(self):
        neighbors = self.get_neighbor_coords()
        return

    def get_neighbor_coords(self):
        row = self.row
        col = self.col
        neighbor_coords = [
            (row - 1, col - 1),
            (row - 1, col),
            (row - 1, col + 1),
            (row, col - 1),
            (row, col + 1),
            (row + 1, col - 1),
            (row + 1, col),
            (row + 1, col + 1),
        ]
        neighbor_coords = [
            (r, c) for r, c in neighbor_coords if 0 <= r <= 9 and 0 <= c <= 9
        ]
        return neighbor_coords

    def set_neighbors(self, octopi):
        neighbor_coords = self.get_neighbor_coords()
        self.neighbors = [x for x in octopi if (x.row, x.col) in neighbor_coords]
        return

    def increase_nrg(self):
        self.nrg_level += 1
        return

    def light_up(self):
        if self.nrg_level > 9 and not self.has_fired:
            self.has_fired = True
            self.total_flashes += 1

            for neighbor in self.neighbors:
                self.propogate(neighbor)

        return

    def propogate(self, other):
        other.nrg_level += 1
        if other.nrg_level > 9 and not other.has_fired:
            other.light_up()

    def reset_nrg(self):
        if self.has_fired is True:
            self.nrg_level = 0
            self.has_fired = False
        return


@dataclass
class Puzzle:
    fname: str
    octopi: list[Octopus] = field(default_factory=list, repr=False)
    n: int = 0
    after_100: int = None
    first_sync: int = None

    def __post_init__(self):
        raw = Path(self.fname).open().readlines()
        levels = [[int(x) for x in line.strip()] for line in raw]

        octopi = []
        for row_num, row in enumerate(levels):
            for col_num, lvl in enumerate(row):
                octopi.append(Octopus(row_num, col_num, lvl))

        self.octopi = octopi
        for octopus in self.octopi:
            octopus.set_neighbors(octopi)
        return

    def single_round(self, part_2=False):

        for octps in self.octopi:
            octps.increase_nrg()

        for octps in self.octopi:
            octps.light_up()

        if self.n == 99:
            self.after_100 = sum([octps.total_flashes for octps in self.octopi])

        if part_2:
            total_diff_nrg = len(set(x.nrg_level for x in self.octopi))
            if total_diff_nrg == 1:
                self.first_sync = self.n

        for octps in self.octopi:
            octps.reset_nrg()
        self.n += 1
        return

    def part_1(self):
        for _ in range(100):
            self.single_round()

        return self.after_100

    def part_2(self):
        while self.first_sync is None:
            self.single_round(part_2=True)
            if self.n > 1000:
                print("This is going too long.")
                break

        return self.first_sync


def run_tests(p1_ans=T1_ANS, p2_ans=T2_ANS, fname="tests.txt"):
    puz = Puzzle(fname)
    t1 = puz.part_1()
    assert t1 == p1_ans, f"Test 1 failed. Got {t1} instead of {p1_ans}"

    if p2_ans is not None:
        t2 = puz.part_2()
        assert t2 == p2_ans, f"Test 2 failed. Got {t2} instead of {p2_ans}"

    print("All tests passed.")
    return


if __name__ == "__main__":
    run_tests()

    puz = Puzzle("inputs.txt")

    p1 = puz.part_1()
    print("Part 1:", p1)

    if T2_ANS is not None:
        p2 = puz.part_2()
        print("Part 2:", p2)

All tests passed.
Part 1: 1615
Part 2: 249


In [ ]:
import numpy as np
from scipy import ndimage

# floor = np.array(
#     [
#         [2, 1, 9, 9, 9, 4, 3, 2, 1, 0],
#         [3, 9, 8, 7, 8, 9, 4, 9, 2, 1],
#         [9, 8, 5, 6, 7, 8, 9, 8, 9, 2],
#         [8, 7, 6, 7, 8, 9, 6, 7, 8, 9],
#         [9, 8, 9, 9, 9, 6, 5, 6, 7, 8],
#     ]
# )

floor = puz.grid

mask = np.array([[0, 1, 0], [1, 1, 1], [0, 1, 0]])

window_minima = ndimage.minimum_filter(floor, footprint=mask, mode="constant", cval=9)
minima = floor[floor == window_minima]
sum(minima + 1)

In [ ]:
np.where(floor == window_minima, 1, 0)